### 최적의 조합
**`제출 기준`**  
- kf-deberta, roberta-large, kr-electra, electra-kor  
  - validation pearson: 0.9371  
  - private: 0.9276  
  - public: 0.9342  

**`아쉽게 제출하지 못한 것`**  
- kf-deberta, kr-electra, ko-electra  
  - validation pearson: 0.9390  
- kf-deberta, roberta-large, kr-electra, ko-electra  
  - validation pearson: 0.9396  

### 앙상블 사용법
클래스는 `ensemble.py`에서 확인  
최종 선정된 모델은 [Google Drive](https://drive.google.com/drive/folders/1BbR8wUIGyn3wSNl3zUOZ-GwbGzURYqrp?usp=sharing)에서 다운 받기  

**`config_paths`**  
fine-tuning 시 사용했던 config.yaml의 경로를 string list로 입력  
이때 모델과 config.yaml은 해당 위치의 `models`라는 폴더 안에 넣어두기(구글 드라이브에서 받은 폴더를 해당 dir에 바로 덮어씌워도 됨)  
config.yaml의 model_name에서 배포자명을 제외한 모델의 이름과 저장한 pt 파일의 이름이 일치하도록 하기(문제가 된다면 수정 예정)  
- 예:  
  ```yaml
  model_name: klue/roberta-small
  ...
  ```
  위와 같이 지정된 경우, 저장된 모델의 이름은 `roberta-small.pt`

**`base_predictions()`**  
fine-tuning을 수행한 LLM들을 사용해서 base predictions를 추출  
데이터 개수만큼의 행, 모델 수만큼의 열이 생성  

**`stacking`, `kfold_stacking`, `soft_voting`**  
각각 기본 stacking, soft voting, kfold stacking으로 메타 모델을 학습  
clf 인자에 'linear'를 입력하면 LinearRegression을, 'lgbm'을 입력하면 LightBGM 모델을 사용하고 클래스의 멤버 변수로 등록  
clf를 바꾸고 싶다면 clf 인자를 변경해서 함수를 다시 호출하면 됨  
kfold stacking은 원래 kfold를 적용한 데이터로 LLM부터 학습해야 하지만 시간 관계 상, 리소스 관계 상 메타 모델에만 k-fold를 적용하는 방식으로 구현  
n 값으로 몇 개의 fold를 사용할지 지정  

**`inference`**  
sample_submission.csv를 읽어와 앙상블 결과 저장  
is_voting이 True이면 soft voting 결과를 사용하고, False이면 stacking과 kfold_stacking 중 가장 마지막에 사용한 classifier를 기준으로 추론 수행  

**`simulate`**  
임의로 두 개의 문장을 생성하여 추론을 수행하면 라벨 값이 출력  
마찬가지로 is_voting이 True이면 voting의 결과를 사용하고, False이면 stacking과 kfold stacking 중 가장 마지막에 사용한 classifier를 기준으로 수행  
앙상블에 사용한 모델의 base prediction과 최종 앙상블 결과가 함께 반환  
이 때 base prediction은 config_paths에서 지정한 순서와 같은 순서로 저장됨  

In [1]:
# roberta base 제외 - 쩔수 없었음;;
config_paths_1 = ['./models/roberta_large.yaml',
                  './models/kf_deberta_cross_sts_config.yaml',
                  './models/KR-ELECTRA-discriminator.yaml',
                  './models/electra-kor-base.yaml']

# 전부 때려 박기
config_paths_2 = ['./models/roberta-base.yaml',
                  './models/roberta_large.yaml',
                  './models/kf_deberta_cross_sts_config.yaml',
                  './models/KR-ELECTRA-discriminator.yaml',
                  './models/electra-kor-base.yaml']

# 다양한 모델, valid pearson 높았던 것 기준
config_paths_3 = ['./models/kf_deberta_cross_sts_config.yaml',
                  './models/electra-kor-base.yaml',
                  './models/roberta_large.yaml']

# 2와 반대의 조합
config_paths_4 = ['./models/kf_deberta_cross_sts_config.yaml',
                  './models/KR-ELECTRA-discriminator.yaml',
                  './models/roberta-base.yaml']

config_paths_5 = ['./models/roberta_large.yaml',
                  './models/kf_deberta_cross_sts_config.yaml',
                  './models/ko_electra_config.yaml',
                  './models/electra-kor-base.yaml']

config_paths_6 = ['./models/ko_electra_config.yaml',
                  './models/kf_deberta_cross_sts_config.yaml',
                  './models/electra-kor-base.yaml']

In [7]:
from ensemble import Ensemble

ensemble = Ensemble(config_paths=config_paths_5,
                    train_path='../../train_preprocess_v1.csv',
                    valid_path='../../dev_preprocess_v1.csv',
                    test_path='../../test_preprocess_v1.csv')

print(ensemble.X_train_base.shape, ensemble.y_train.shape)
print(ensemble.X_valid_base.shape, ensemble.y_valid.shape)
print(ensemble.X_test_base.shape)

========== base predicting train data ==========
+++++ Right now using "klue/roberta-large" +++++


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
base prediction: 100%|██████████| 583/583 [04:39<00:00,  2.08it/s]


+++++ Right now using "deliciouscat/kf-deberta-base-cross-sts" +++++


base prediction: 100%|██████████| 583/583 [02:13<00:00,  4.37it/s]


+++++ Right now using "monologg/koelectra-base-v3-discriminator" +++++


...

+++++ Right now using "monologg/koelectra-base-v3-discriminator" +++++


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
base prediction: 100%|██████████| 1100/1100 [00:16<00:00, 67.09it/s]


+++++ Right now using "kykim/electra-kor-base" +++++


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
base prediction: 100%|██████████| 1100/1100 [00:16<00:00, 67.46it/s]



(9324, 4) (9324, 1)
(550, 4) (550, 1)
(1100, 4)


In [8]:
ensemble.stacking(clf="linear")
ensemble.stacking(clf="xgboost")
ensemble.soft_voting()
ensemble.kfold_stacking("linear", 3)
ensemble.kfold_stacking("xgboost", 3)

========== linear stacking result ==========
    train pearson sim: [0.99909038]
    valid pearson sim: [0.93723378]


========== xgboost stacking result ==========
    train pearson sim: [0.99867074]
    valid pearson sim: [0.92889521]


========== soft voting result ==========
    train pearson sim: [0.99874349]
    valid pearson sim: [0.93958898]


========== linear stacking result ==========
    train pearson sim: [0.99908937]
    valid pearson sim: [0.93706956]


========== xgboost stacking result ==========
    train pearson sim: [0.99842974]
    valid pearson sim: [0.92431907]




(array([0.99842974]), array([0.92431907]))

In [6]:
# ensemble.stacking("linear")
ensemble.inference(is_voting=True, submission_path='../../sample_submission.csv')

saved ensemble_output.csv


In [14]:
import numpy as np

sentence_1 = "손 틈 새로 비치는"
sentence_2 = "아이유 참 좋다"
voting_base, voting_ensemble = ensemble.simulate(sentence_1, sentence_2, True)
_, stacking_ensemble = ensemble.simulate(sentence_1, sentence_2, False)

print(f"base predictions: {np.round(voting_base, 1)}, voting prediction: {voting_ensemble}, stacking prediction: {stacking_ensemble}")

Now Using deliciouscat/kf-deberta-base-cross-sts
Now Using snunlp/KR-ELECTRA-discriminator


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Now Using deliciouscat/kf-deberta-base-cross-sts
Now Using snunlp/KR-ELECTRA-discriminator
base predictions: [[ 0.1 -0. ]], voting prediction: [0.], stacking prediction: [1.7]
